In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
from scipy import stats
from matplotlib import pyplot as plt
import sklearn as sk
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [37]:
import requests
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")

In [38]:
URL = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-11-03/ikea.csv'

req = requests.get(URL)

In [39]:
req.headers

{'Connection': 'keep-alive', 'Content-Length': '154500', 'Cache-Control': 'max-age=300', 'Content-Security-Policy': "default-src 'none'; style-src 'unsafe-inline'; sandbox", 'Content-Type': 'text/plain; charset=utf-8', 'ETag': 'W/"d901e6fe53bcc43168991cd6d6156d10a1dc274f9088d941dfbe6520d54137fc"', 'Strict-Transport-Security': 'max-age=31536000', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'deny', 'X-XSS-Protection': '1; mode=block', 'X-GitHub-Request-Id': '57A0:421B:E92970:F2986A:64D4AF6A', 'Content-Encoding': 'gzip', 'Accept-Ranges': 'bytes', 'Date': 'Thu, 10 Aug 2023 09:35:39 GMT', 'Via': '1.1 varnish', 'X-Served-By': 'cache-vie6322-VIE', 'X-Cache': 'MISS', 'X-Cache-Hits': '0', 'X-Timer': 'S1691660139.223803,VS0,VE232', 'Vary': 'Authorization,Accept-Encoding,Origin', 'Access-Control-Allow-Origin': '*', 'Cross-Origin-Resource-Policy': 'cross-origin', 'X-Fastly-Request-ID': 'baca59994fcae48d6695115e79400669c8f10871', 'Expires': 'Thu, 10 Aug 2023 09:40:39 GMT', 'Source-Age':

In [40]:
match req.status_code:
    case 200: 
        df = pd.read_csv(URL)
    case 404: 
        raise 'Problem with connection'
    case _ :
        raise f'Something wrong {req.status_code}'

**EDA**

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.columns

In [ ]:
#Size of dataset
df.shape

In [ ]:
df.dtypes

In [ ]:
#count how many times each data type is present in the dataset
pd.value_counts(df.dtypes)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

We have 3 features with NULL values: **depth**, **height**, **width** . When we do the ML, we will need to fill them in.

In [ ]:
df['category'].unique()

In [ ]:
df['category'].value_counts()

In [ ]:
df.head()

In [ ]:
df['sellable_online'].value_counts()

In [ ]:
df['category'].value_counts(normalize= True)

In [76]:
# Create copy of df, for delete null 
df1 = df.copy()

df1.dropna(inplace= True)

In [ ]:
df1.head()

We can see to extra columns: **item_id**, **link**. We will remove them, as they are not important in the analysis.

In [ ]:
df1 = df1.drop(['item_id', 'link'], axis= 1)
df1.columns

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
plt.hist(df1['category'],bins = 27, color = 'red')

plt.title('Розподіл для category')
plt.xlabel('category')
plt.ylabel('Частота')
plt.show()

In [ ]:
df1['category'].unique()

In [83]:
df1['category'].nunique()

17

In [ ]:
df1.sort_values('price', ascending= False)

In [ ]:
df1.sort_values(['price', 'category'], ascending= [False,True])

In [ ]:
df1.duplicated().sum()

In [ ]:
df1.groupby('category')['price'].mean()

In [ ]:
df1.groupby('price')['old_price'].count()

In [ ]:
df1.groupby('old_price')['category'].count()

In [ ]:
df1.loc[0:10,['category','designer']]

In [ ]:
df1.iloc[0:10,1:3]

In [ ]:
df1.head()

In [ ]:
sns.barplot(data=df1, x="price", y="category", palette= 'magma')
plt.grid()
plt.xlabel('price')
plt.ylabel('category')
plt.title('category by price')
plt.show();

In [ ]:
sns.barplot(data=df1, x="depth", y="category", palette= 'magma')
plt.grid()
plt.xlabel('depth')
plt.ylabel('category')
plt.title('category by depth')
plt.show();

In [ ]:
plt.figure(figsize = (6,4)) 

sns.countplot(y= df1.category ,data = df); 

plt.xlabel('count') 
plt.ylabel('category') 
plt.title('Ikea') 
plt.show() 

In [ ]:
sns.heatmap(df1.corr(),annot= True, cmap='magma');

In [ ]:
sns.heatmap(df[['price', 'depth', 'height', 'width']].corr(), xticklabels= df[['price', 'depth', 'height', 'width']].corr().columns,
            yticklabels= df[['price', 'depth', 'height', 'width']].corr().columns, center= 0, annot= True);

From the correlation graph, we can conclude that the **most correlated features** are *price* and *width*; *price* and *depth*

In [ ]:
df1.head()

In [ ]:
sns.scatterplot(x = 'price', y = 'width', data = df1);
plt.grid(color = 'black', linewidth = 0.8)

In [ ]:
sns.scatterplot(x = 'price', y = 'depth', data = df1);
plt.grid(color = 'black', linewidth = 0.8)

In [ ]:
plt.subplots(figsize = (8,6))
sns.scatterplot(x = 'price', y = 'category', data = df1, hue = 'sellable_online');
plt.grid()

In [ ]:
sns.pairplot(df[['price', 'depth', 'height', 'width', 'other_colors', 'category']], hue = 'other_colors');

In [101]:
df_X = df1[['height', 'width', 'depth', 'price']]

In [ ]:
# Take SelectKbest to find more informative columns
# chi2 - 
from sklearn.feature_selection import chi2, SelectKBest, mutual_info_classif, mutual_info_regression
_, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,8))

w_df_X = df1[['height', 'width', 'depth', 'price']]
w_df_y = df1['category']

# descrete 'category'
importance_0 = mutual_info_classif(w_df_X, w_df_y)
feature_importances_0 = pd.Series(importance_0, w_df_X.columns)
feature_importances_0.plot(kind='barh', color='red', ax = ax1)

w_df_X_1 = df1[['height', 'width', 'depth']]
w_df_y_1 = df1['price']

# long 'price'
importance_1 = mutual_info_regression(w_df_X_1, w_df_y_1)
feature_importances_1 = pd.Series(importance_1, w_df_X_1.columns)
feature_importances_1.plot(kind='barh', color='purple', ax = ax2)

chi2_features = SelectKBest(chi2, k=3) 
X_kbest_features = chi2_features.fit(w_df_X, w_df_y)
print('Best features:', X_kbest_features.get_support(indices=True))

transformed_X = chi2_features.transform(w_df_X)
print(transformed_X)

plt.show() # See that width and height important columns

In [ ]:
df['designer'].head(15)

In [106]:
# Create function to clear Designers
def cleanDesigners(value, removeIKEA=False, emptyValue=np.nan):
    if not isinstance(value, str):
        return value
    
    if len(value)>0 and value[0].isdigit():
        return emptyValue
    
    designers = value.split("/")
    
    if removeIKEA:
        try:
            designers.remove("IKEA of Sweden")
        except:
            pass
    if len(designers) > 0:
        return '/'.join(sorted(designers))
    else:
        return emptyValue

In [107]:
ikea_df = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-11-03/ikea.csv", index_col= 0).drop_duplicates()

ikea_df['designer_clean'] = df['designer'].apply(cleanDesigners, args= (False, "IKEA of Sweden"))

In [ ]:
ikea_df['designer_clean'].head()

In [ ]:
ikea_df.isnull().sum()

In [ ]:
# Fill in data
ikea_df['other_colors_1'] = ikea_df['other_colors'].map(dict(Yes=1, No=0))

median_d = ikea_df.groupby(['category'])['depth'].median()
median_h = ikea_df.groupby(['category'])['height'].median()
median_w = ikea_df.groupby(['category'])['width'].median()

median_price = ikea_df.groupby(['category'])['price'].median()
median_dsgn =  ikea_df.groupby(['designer_clean'])['price'].median()

ikea_df = ikea_df.set_index(['category'])
ikea_df['depth_1'] = ikea_df['depth'].fillna(median_d)
ikea_df['height_1'] = ikea_df['height'].fillna(median_h)
ikea_df['width_1'] = ikea_df['width'].fillna(median_w)
ikea_df['category_median_price'] = median_price

ikea_df = ikea_df.reset_index()

ikea_df = ikea_df.set_index(['designer_clean'])
ikea_df['designer_median_price'] = median_dsgn
ikea_df = ikea_df.reset_index()
ikea_df.head()

In [123]:
# Function to choose BestRegressor
# Step by step fit data for different model, and display result
def getBestRegressor (X, Y):
    X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)
    models = [
        sk.linear_model.LinearRegression(),
        sk.linear_model.LassoCV(),
        sk.linear_model.RidgeCV(),
        sk.svm.SVR(kernel='linear'),
        sk.neighbors.KNeighborsRegressor(n_neighbors=16),
        sk.tree.DecisionTreeRegressor(max_depth=10, random_state=42),
        RandomForestRegressor(random_state=42),
        GradientBoostingRegressor()
    ]
    
    TestModels = pd.DataFrame()
    res = {}
    tmp = {}
    
    for model in models:
        m = str(model)
        tmp['Model'] = m[:m.index('(')]
        model.fit(X_train, Y_train)
        tmp['R^2'] = '{:.5f}'.format(model.score(X_test, Y_test))
        tmp['MAE'] = '{:.5f}'.format(sk.metrics.mean_absolute_error(model.predict(X_test), Y_test))
        tmp['RMSE'] = '{:.5f}'.format(np.sqrt(sk.metrics.mean_squared_error(model.predict(X_test), Y_test)))
        
        TestModels = pd.concat([TestModels, pd.DataFrame([tmp])])
        
    TestModels.set_index('Model', inplace=True)
    res['model'] = TestModels
    res['X_train'] = X_train
    res['Y_train'] = Y_train
    res['X_test'] = X_test
    res['Y_test'] = Y_test
    return res

In [ ]:
X1 = ikea_df[['width_1','depth_1','height_1']]
Y1 = ikea_df['price']

test1 = getBestRegressor(X1, Y1)
test1['model'].sort_values(by='R^2', ascending=False)


We can see that **RandomForestRegressor** have best result for predict price, next step to tunning model 

In [ ]:
from sklearn.model_selection import GridSearchCV
# Use  GridSearchCV for tunning model 
X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(X1, Y1, test_size=0.2, random_state=42)
forest_grid = GridSearchCV(RandomForestRegressor(), {'n_estimators': [10,25,50,100,110,120,130, 140,150,200,500], 
                                                     'max_depth': [10,20,30,40,50,100,200,None], 
                                                     'max_features': ['auto','sqrt','log2']}, cv=5, n_jobs=-1, verbose=0)
forest_grid.fit(X_train, Y_train)
 
print('Best Estimator :',forest_grid.best_estimator_)
print('Best Score     :',forest_grid.best_score_)
print('')
print('R^2            : {:.5f}'.format(sk.metrics.r2_score(Y_test, forest_grid.predict(X_test))))
print('MAE            : {:.5f}'.format(sk.metrics.mean_absolute_error(forest_grid.predict(X_test), Y_test)))
print('RMSE           : {:.5f}'.format(np.sqrt(sk.metrics.mean_squared_error(forest_grid.predict(X_test), Y_test))))
print('')
print('Feature importance:')
print('--------------------------------')

for feat, importance in zip(X_train.columns, forest_grid.best_estimator_.feature_importances_):
    print('{:.5f}    {f}'.format(importance, f=feat))

In [ ]:
sns.set_style('whitegrid')
sns.barplot(y=X_train.columns, x=forest_grid.best_estimator_.feature_importances_)

After analyzing and machine learning, we determined that the most important features for pricing in our dataset are: **width, depth and height**.